# Scrape Combined ND Football Rosters From Sports Reference



# Load Modules Using Import

In [ ]:
# import requests
import requests

# import csv
import csv

# import beautifulsoup
from bs4 import BeautifulSoup
from bs4 import Comment

# import pandas
import pandas as pd

# import numpy
import numpy as np

# Get List of Season URLs from Sports Reference

In [ ]:
# root url
root = "https://www.sports-reference.com/cfb/schools/notre-dame/"

# year range
years = range(1924, 2021, 1)

# url tag
tag = ".html"

# empty list for urls
urls = []

# for loop that concatenates full url
for year in years:
    urls.append(root + str(year) + tag)

# show list of urls
urls

# Create Dictionary That Connects Season Year With URL

In [ ]:
test_dict = {years[i]: urls[i] for i in range(len(years))}

test_dict

# Create Dictionary of Pandas DataFrames with Each Year's Roster

In [ ]:
# empty dictionary
rosters = {}

# for loop that loads roster table as pandas data frame and saves dataframe as dictionary value associated with season year key; also includes data cleaning lines within for loop
for key, value in test_dict.items():
    df_name = key
    try:
        page = requests.get(value)
        soup = BeautifulSoup(page.text, 'html.parser')
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        rushing_stats = comments[31]
        test = BeautifulSoup(rushing_stats, 'html.parser')
        table = test.find("table")
        dfs = pd.read_html(str(table))
        df = dfs[0]
        df.columns = [''.join(col).strip() for col in df.columns.values]
        df.rename(columns = {'Unnamed: 0_level_0Rk': 'Rk', 'Unnamed: 1_level_0Player': 'Player', 'Unnamed: 2_level_0G': 'G'}, inplace=True)
        df.loc[:, 'Season'] = key # create season column with dictionary key
        rosters[df_name] = df
    except:
        continue

# show dictionary
rosters

# Combine Dictionary of DataFrames using `pd.concat`

In [ ]:
# create combined_schedule df
combined_roster = pd.concat(rosters.values(), ignore_index=True)

# show df
combined_roster

In [ ]:
# create first and last name columns
combined_roster[['First_Name', 'Last_Name']] = combined_roster.Player.str.split(" ", 1, expand=True)

# show updated dataframe
combined_roster

In [ ]:
# list with desired column order
column_names = ['Rk', 'Season', 'Player', 'First_Name', 'Last_Name', 'G', 'RushingAtt', 'RushingYds', 'RushingAvg', 'RushingTD', 'ReceivingRec', 'ReceivingYds', 'ReceivingAvg', 'ReceivingTD', 'ScrimmagePlays', 'ScrimmageYds', 'ScrimmageAvg', 'ScrimmageTD']

# reorder columns
combined_roster = combined_roster.reindex(columns = column_names)

# show updated dataframe
combined_roster

# Write DataFrame to CSV File

In [ ]:
# write combined_roster dataframe to csv file
combined_roster.to_csv("combined_nd_rosters.csv", index=False)